In [1]:
import polars as pl

pl.Config.set_tbl_cols(-1)  # Show all columns
pl.Config.set_tbl_width_chars(10000)  # Set table width
pl.Config.set_fmt_str_lengths(10000)  # Set max string length
pl.Config.set_tbl_rows(32)  # Show all rows

# Read CSV data as a dataframe

games = pl.read_csv("games.csv")
print(games)

shape: (194, 4)
┌──────────────────────┬────────────┬───────────────────────┬────────────┐
│ away_team            ┆ away_score ┆ home_team             ┆ home_score │
│ ---                  ┆ ---        ┆ ---                   ┆ ---        │
│ str                  ┆ i64        ┆ str                   ┆ i64        │
╞══════════════════════╪════════════╪═══════════════════════╪════════════╡
│ Dallas Cowboys       ┆ 20         ┆ Philadelphia Eagles   ┆ 24         │
│ Kansas City Chiefs   ┆ 21         ┆ Los Angeles Chargers  ┆ 27         │
│ Arizona Cardinals    ┆ 20         ┆ New Orleans Saints    ┆ 13         │
│ Pittsburgh Steelers  ┆ 34         ┆ New York Jets         ┆ 32         │
│ Miami Dolphins       ┆ 8          ┆ Indianapolis Colts    ┆ 33         │
│ Tampa Bay Buccaneers ┆ 23         ┆ Atlanta Falcons       ┆ 20         │
│ New York Giants      ┆ 6          ┆ Washington Commanders ┆ 21         │
│ Carolina Panthers    ┆ 10         ┆ Jacksonville Jaguars  ┆ 26         │
│ Cincinn

In [2]:
homeResults = games.select(
    team=pl.col("home_team"),
    scored=pl.col("home_score"),
    allowed=pl.col("away_score"),
)

awayResults = games.select(
    team=pl.col("away_team"),
    scored=pl.col("away_score"),
    allowed=pl.col("home_score"),
)

# Combine the home and away win loss results

combined = pl.concat([homeResults, awayResults])
print(combined)

shape: (388, 3)
┌───────────────────────┬────────┬─────────┐
│ team                  ┆ scored ┆ allowed │
│ ---                   ┆ ---    ┆ ---     │
│ str                   ┆ i64    ┆ i64     │
╞═══════════════════════╪════════╪═════════╡
│ Philadelphia Eagles   ┆ 24     ┆ 20      │
│ Los Angeles Chargers  ┆ 27     ┆ 21      │
│ New Orleans Saints    ┆ 13     ┆ 20      │
│ New York Jets         ┆ 32     ┆ 34      │
│ Indianapolis Colts    ┆ 33     ┆ 8       │
│ Atlanta Falcons       ┆ 20     ┆ 23      │
│ Washington Commanders ┆ 21     ┆ 6       │
│ Jacksonville Jaguars  ┆ 26     ┆ 10      │
│ Cleveland Browns      ┆ 16     ┆ 17      │
│ New England Patriots  ┆ 13     ┆ 20      │
│ Seattle Seahawks      ┆ 13     ┆ 17      │
│ Denver Broncos        ┆ 20     ┆ 12      │
│ Los Angeles Rams      ┆ 14     ┆ 9       │
│ Green Bay Packers     ┆ 27     ┆ 13      │
│ Buffalo Bills         ┆ 41     ┆ 40      │
│ Chicago Bears         ┆ 24     ┆ 27      │
│ …                     ┆ …      ┆ …   

In [3]:
# Group the stats from each team game into a single row

summary = combined.group_by("team").agg(
    numGames=pl.len(),
    wins=(pl.col("scored") > pl.col("allowed")).sum(),
    losses=(pl.col("scored") < pl.col("allowed")).sum(),
    ties=(pl.col("scored") == pl.col("allowed")).sum(),
    scored=pl.sum("scored"),
    allowed=pl.sum("allowed"),
)

print(summary)

shape: (32, 7)
┌───────────────────────┬──────────┬──────┬────────┬──────┬────────┬─────────┐
│ team                  ┆ numGames ┆ wins ┆ losses ┆ ties ┆ scored ┆ allowed │
│ ---                   ┆ ---      ┆ ---  ┆ ---    ┆ ---  ┆ ---    ┆ ---     │
│ str                   ┆ u32      ┆ u32  ┆ u32    ┆ u32  ┆ i64    ┆ i64     │
╞═══════════════════════╪══════════╪══════╪════════╪══════╪════════╪═════════╡
│ Denver Broncos        ┆ 12       ┆ 10   ┆ 2      ┆ 0    ┆ 284    ┆ 218     │
│ Houston Texans        ┆ 12       ┆ 7    ┆ 5      ┆ 0    ┆ 263    ┆ 198     │
│ Kansas City Chiefs    ┆ 12       ┆ 6    ┆ 6      ┆ 0    ┆ 305    ┆ 232     │
│ New York Giants       ┆ 13       ┆ 2    ┆ 11     ┆ 0    ┆ 279    ┆ 367     │
│ Cleveland Browns      ┆ 12       ┆ 3    ┆ 9      ┆ 0    ┆ 194    ┆ 270     │
│ New England Patriots  ┆ 13       ┆ 11   ┆ 2      ┆ 0    ┆ 351    ┆ 241     │
│ Baltimore Ravens      ┆ 12       ┆ 6    ┆ 6      ┆ 0    ┆ 289    ┆ 293     │
│ Detroit Lions         ┆ 12       ┆ 

In [4]:
# Calculate the per-game stats for each team
# Sort the team data by win percentage, highest to lowest

finalStats = summary.with_columns(
    winPct=((pl.col("wins") + 0.5 * pl.col("ties")) / pl.col("numGames")).round(3),
    scoredPerGame=(pl.col("scored") / pl.col("numGames")).round(1),
    allowedPerGame=(pl.col("allowed") / pl.col("numGames")).round(1),
).sort("winPct", descending=True)

print(finalStats)

shape: (32, 10)
┌───────────────────────┬──────────┬──────┬────────┬──────┬────────┬─────────┬────────┬───────────────┬────────────────┐
│ team                  ┆ numGames ┆ wins ┆ losses ┆ ties ┆ scored ┆ allowed ┆ winPct ┆ scoredPerGame ┆ allowedPerGame │
│ ---                   ┆ ---      ┆ ---  ┆ ---    ┆ ---  ┆ ---    ┆ ---     ┆ ---    ┆ ---           ┆ ---            │
│ str                   ┆ u32      ┆ u32  ┆ u32    ┆ u32  ┆ i64    ┆ i64     ┆ f64    ┆ f64           ┆ f64            │
╞═══════════════════════╪══════════╪══════╪════════╪══════╪════════╪═════════╪════════╪═══════════════╪════════════════╡
│ New England Patriots  ┆ 13       ┆ 11   ┆ 2      ┆ 0    ┆ 351    ┆ 241     ┆ 0.846  ┆ 27.0          ┆ 18.5           │
│ Denver Broncos        ┆ 12       ┆ 10   ┆ 2      ┆ 0    ┆ 284    ┆ 218     ┆ 0.833  ┆ 23.7          ┆ 18.2           │
│ Chicago Bears         ┆ 12       ┆ 9    ┆ 3      ┆ 0    ┆ 313    ┆ 307     ┆ 0.75   ┆ 26.1          ┆ 25.6           │
│ Los Angeles Ra